# "Poker Combinatorics with Itertools"

- toc: false
- comments: false
- permalink: /combos
- categories: [poker, python]

Suppose you're playing No Limit Hold'em, and based on how your opponent has played the hand, you believe they have pocket aces, pocket kings, or ace-king. How likely is each hand? (Hint: They're not equally likely!)

The math that determines the likelihood of each hand is called combinatorics and it's in a sweet spot of being non-obvious, yet pretty easy to understand, and highly relevant for in-game strategy. I just finished [Fluent Python](https://learning.oreilly.com/library/view/fluent-python/9781491946237/) by Luciano Ramalho and got a refresher on the [Python built-in itertools module](https://docs.python.org/3/library/itertools.html). It's the perfect way to explore poker combinations.

By the way, you can run this post as a colab notebook. Click the colab link under the title to open a notebook in colab where you can run the code or try your own changes.

In [96]:
import itertools

The first example in the book is actually a playing card deck, so we can borrow a little code to get started.

In [98]:
suits = 'spades hearts diamonds clubs'.split()
ranks = [str(n) for n in range(2, 11)] + list('JQKA')

To get the cards in the deck, you can use the `product` function from itertools. Here's the description:

"Cartesian product: yields N-tuples made by combining items from each input iterable like nested for loops could produce."

In [103]:
cards = list(itertools.product(ranks, suits))

All the itertools functions return iterators, so if you want to do things like check the length, you need to coerce the result into a list.

In [111]:
len(cards)

52

In [112]:
cards[0], cards[-1]

(('2', 'spades'), ('A', 'clubs'))

Story checks out!

Now, combinations (or combos) are all the ways you can deal a two-card starting hand. Conveniently, itertools has a `combinations` function.

In [113]:
combos = list(itertools.combinations(cards, 2))

In [114]:
len(combos)

1326

There are 1326 combinations, but these include combinations that you would typically consider the same hand.

In [137]:
combos[0]

(('2', 'spades'), ('2', 'hearts'))

In [138]:
combos[1]

(('2', 'spades'), ('2', 'diamonds'))

Even though the above hands have different cards, you'd probably think of both of them as "pocket deuces."

If you play a lot of poker, you've likely seen "The Grid," which is a more intuitive way of representing hands. 

![poker grid](combos/poker_grid.png)

In this way of representing the hands, the upper right hands are suited, the lower left are offsuit, and the pairs go diagonally across the middle.

To get something more like the grid representation of hands, you can take the product of ranks with itself.

In [117]:
hands = list(itertools.product(ranks, ranks))

In [118]:
len(hands)

169

In [119]:
hands[0]

('2', '2')

In [120]:
hands[-1]

('A', 'A')

How likely a hand is depends on how many ways it can be dealt - the combinations. You now have all the information you need to answer the initial question: If you think your opponent has pocket aces, pocket kings, or ace-king, how likely is each?

In [125]:
aces = [('A', s) for s in suits]
kings = [('K', s) for s in suits]

In [126]:
AA = list(itertools.combinations(aces, 2))
KK = list(itertools.combinations(kings, 2))

In [127]:
len(AA)

6

In [128]:
AA

[(('A', 'spades'), ('A', 'hearts')),
 (('A', 'spades'), ('A', 'diamonds')),
 (('A', 'spades'), ('A', 'clubs')),
 (('A', 'hearts'), ('A', 'diamonds')),
 (('A', 'hearts'), ('A', 'clubs')),
 (('A', 'diamonds'), ('A', 'clubs'))]

There are six ways to deal pocket aces. The same applies for pocket kings, or any other pair, of course. The chances of being dealt pocket aces are six divided by the total number of combinations.

In [131]:
len(AA) / len(combos)

0.004524886877828055

You get dealt pocket aces about one in 200 hands.

In [132]:
AK = list(itertools.product(aces, kings))

In [133]:
len(AK)

16

There are 16 combinations of AK. Of those, some are suited and some are offsuit.

In [134]:
AKs = [h for h in AK if h[0][1] == h[1][1]]
AKo = [h for h in AK if h[0][1] != h[1][1]]

In [135]:
len(AKs), len(AKo)

(4, 12)

Given that there are 16 combos of ace-king and only six each of aces and kings, ace-king is more likely than aces and kings *combined*. Given that pocket deuces is a slight favorite against ace-king, if you hold deuces in this situation, you are actually ahead more often than not. Unfortunately, this is a classic slightly ahead/way behind scenario: you're either a tiny favorite or a huge underdog, so you're still basically screwed. But that's a topic for another day.

## Takeaways
* Not all hands are equally likely.
* Offsuit, unpaired hands are the most likely. Pairs are less likely. Suited hands are the rarest are all.
* This means, if you are considering how likely it is for your opponent to have a certain hand, it's *really* important whether or not they would play the offsuit flavor of that hand.